# Track all models in a model registry

In this section we will walk through how you can register your model to the SageMaker Model Registry. 

The SageMaker Model Registry is structured as several model groups with model packages in each group. Each model package in a model group corresponds to a trained model. The version of each model package is a numerical value that starts at 1 and is incremented with each new model package added to a model group. For example, if 5 model packages are added to a model group, the model package versions will be 1, 2, 3, 4, and 5. The example Model Registry shown in the following image contains 3 model groups, where each group contains the model packages related to a particular ML problem.

In [29]:
!pip install sagemaker==2.117

In [30]:
import sagemaker
import json
import boto3
import numpy as np                                
import pandas as pd                               
import os                                         
from sagemaker import get_execution_role
from datetime import datetime

# Get user profile name
metadataFile = open('/opt/ml/metadata/resource-metadata.json')
metadata = json.load(metadataFile)
userprofileName = metadata['UserProfileName']

# Get default bucket
bucket = sagemaker.Session().default_bucket()
prefix = f'sagemaker/{userprofileName}/mlops-workshop'

# Get SageMaker Execution Role
role = get_execution_role()
region = boto3.Session().region_name

# SageMaker Session
sagemaker_session = sagemaker.session.Session()

### Retrieve variables

In [31]:
%store -r

In [32]:
print(trained_model_uri)
print(training_image)

s3://sagemaker-ca-central-1-222848388999/sagemaker/pplhy997-intact-net-bc8/mlops-workshop/xgb_model/xgboost-2022-12-15-18-37-42-038/output/model.tar.gz
469771592824.dkr.ecr.ca-central-1.amazonaws.com/xgboost:latest


In [33]:
from sagemaker.model import Model

model = Model(
    image_uri = training_image,
    model_data = trained_model_uri,
    role = role,
    sagemaker_session = sagemaker_session
)

### Register Model

In [34]:
model_package = model.register(
    description = "This is a model to detect the liklihood of someone getting approved for a loan",
    model_package_group_name = f"model-group-module-1-{userprofileName}",
    customer_metadata_properties = {
        'ModelType' : 'Decision Tree',
        'Algorithm' : 'XGBoost'
    },
    inference_instances = ['ml.m5.xlarge'],
    approval_status = 'PendingManualApproval',
    content_types = ['text/csv'],
    response_types = ['text/csv']
)

In [35]:
model_package_arn = model_package.model_package_arn

In [36]:
print("Model Package arn: ", model_package_arn)

Model Package arn:  arn:aws:sagemaker:ca-central-1:222848388999:model-package/model-group-module-1-pplhy997-intact-net-bc8/3


In [37]:
%store model_package_arn

Stored 'model_package_arn' (str)


#### You can now move to the next section of the module `Deploy your models for inference`

The notebook used in that section is `sagemaker-deploy.ipynb`